In [12]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [13]:
DROPOUT_VALUE = 0.04

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16)
        )

        self.trans = nn.Sequential(
            nn.Conv2d(16, 8, 1, padding=0, bias=False),
            nn.MaxPool2d(2, 2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(DROPOUT_VALUE)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(DROPOUT_VALUE)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(32, 32, 3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32)
        )

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=6)
        )

        self.conv6 = nn.Sequential(
            nn.Conv2d(32, 10, 1, padding=0, bias=False)
        )

    def forward(self, x):
        device = next(self.parameters()).device  # Get the device of the model
        x = x.to(device)  # Move input tensor to the same device as the model
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.trans(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.gap(x)
        x = self.conv6(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)


In [14]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 16, 24, 24]           1,152
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
            Conv2d-7            [-1, 8, 24, 24]             128
         MaxPool2d-8            [-1, 8, 12, 12]               0
            Conv2d-9           [-1, 16, 10, 10]           1,152
             ReLU-10           [-1, 16, 10, 10]               0
      BatchNorm2d-11           [-1, 16, 10, 10]              32
           Conv2d-12             [-1, 16, 8, 8]           2,304
             ReLU-13             [-1, 16, 8, 8]               0
      BatchNorm2d-14             [-1, 1

In [15]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=False, **kwargs)

In [16]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nEpoch: {},Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [17]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.03460963815450668 batch_id=1874: 100%|██████████| 1875/1875 [00:17<00:00, 105.63it/s]



Epoch: 1,Test set: Average loss: 0.0512, Accuracy: 9839/10000 (98.3900%)



loss=0.006276600528508425 batch_id=1874: 100%|██████████| 1875/1875 [00:17<00:00, 107.17it/s]



Epoch: 2,Test set: Average loss: 0.0291, Accuracy: 9909/10000 (99.0900%)



loss=0.03227299824357033 batch_id=1874: 100%|██████████| 1875/1875 [00:17<00:00, 105.75it/s]


...


Epoch: 17,Test set: Average loss: 0.0184, Accuracy: 9932/10000 (99.3200%)



loss=0.002826014533638954 batch_id=1874: 100%|██████████| 1875/1875 [00:17<00:00, 105.20it/s]



Epoch: 18,Test set: Average loss: 0.0199, Accuracy: 9933/10000 (99.3300%)



loss=0.00019296756363473833 batch_id=1874: 100%|██████████| 1875/1875 [00:18<00:00, 103.34it/s]



Epoch: 19,Test set: Average loss: 0.0178, Accuracy: 9945/10000 (99.4500%)



In [18]:
torch.save(model.state_dict(), '/content/mnist_test_acc_99.45_epochs_19.pth')